<a href="https://colab.research.google.com/github/divya-thota/OCR-Reader/blob/main/OCR_Reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!apt-get install poppler-utils &> /dev/null
!pip install pdf2image &> /dev/null

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [ ]:
import pandas as pd
import cv2 
import pytesseract
from PIL import Image
from pdf2image import convert_from_path

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)
dataframe_papers = gc.open('dataframe_papers').sheet1
staff_master = gc.open('staff_master').sheet1

In [ ]:
xls = pd.ExcelFile('Master.xlsx')
df1 = pd.read_excel(xls, 'PSU CSU', header=None)
df2 = pd.read_excel(xls, 'Non Unit', header=None)
df3 = pd.read_excel(xls, 'FSU', header=None)

df1 = df1.append(df2, ignore_index=True)
df1 = df1.append(df3, ignore_index=True)

df1.columns = df1.iloc[0]
df1 = df1[1:]
df1.drop_duplicates()
df1 = df1.reset_index(drop=True)

#df1 = df1[['First_Name','Last_Name', 'EMPL_UMS_ID','University_Affiliation']]
df1['EMPL_UMS_ID'] = df1.EMPL_UMS_ID.astype(str)
df1['EMPL_UMS_ID'] = df1['EMPL_UMS_ID'].str[2:]
df1["Found"] = pd.NaT

In [ ]:
PDF_file = "d.pdf"
pages = convert_from_path(PDF_file, 500)
image_counter = 1
  
for page in pages:
  filename = "page_"+str(image_counter)+".jpg"
  page.save(filename, 'JPEG')
  image_counter = image_counter + 1

df = pd.DataFrame()
filelimit = image_counter-1
for i in range(1, filelimit + 1):
  filename = "page_"+str(i)+".jpg"
  custom_config = r'--oem 3 --psm 6'
  text = str(pytesseract.image_to_string(filename, config=custom_config))
  df_partial = pd.DataFrame([x.split(' ') for x in text.split('\n')])
  df_partial = df_partial.drop(range(df_partial.index[df_partial[1] == "ID"][0]), axis = 0 )
  df_partial = df_partial.reset_index(drop=True)
  df_partial.columns = df_partial.iloc[0]
  df_partial = df_partial[1:]
  df_partial['Red'] = df_partial['Red'].str.replace('999', '')
  df_partial['Red'] = df_partial['Red'].str.replace(',', '')
  df_partial["Employee Name"] = df_partial["ID"] + " " + df_partial["Name"]  + " " + df_partial["Red"]
  df_partial = df_partial[['ployee', 'Employee Name']]
  df = df.append(df_partial,ignore_index=True)

df = df.dropna()
df = df.reset_index(drop=True)
df

,ployee,Employee Name
0,190678,"Bowen,Marie H"
1,031653,"Bussiere,Elizabeth 999 Le"
2,091117,"Curry, Shaun"
3,186413,"Fuchs,Christopher Al"
4,162750,"N Kamborian, Kristopher"
...,...,...
522,034648,"Zhang, Honggang"
523,256364,"Zhu, Youxiang"
524,254931,"de Lemos, Simone"
525,Deduction,"Total 9, 30375"


In [ ]:
dataframe_papers = pd.DataFrame(columns = ['ID','Name'])
for index,row in df.iterrows():
  if df1['EMPL_UMS_ID'].str.contains(str(row['ployee'])).any():
    df1.loc[df1['EMPL_UMS_ID'] == str(row['ployee']), 'Found'] = "Found"
  else:
    dataframe_papers = dataframe_papers.append({'ID': row['ployee'], 'Name': row['Employee Name']},ignore_index=True)


#dataframe_papers.update([dataframe_papers.columns.values.tolist()] + dataframe_papers.values.tolist())
#staff_master.update([df1.columns.values.tolist()] + df1.values.tolist())

In [ ]:
df1

,Date,Status,First_Name,Last_Name,Email,Phone,on_off_campus,University_Affiliation,Start_Date,Discontinue,...,Affiliation,Full_Pass_Type,License_Plate_1,License_Plate_2,License_Plate_3,Beacon_Card_Number,Photos,Agree_to_Terms,Acknowledgement,Found
0,2022-04-28 00:00:00,Approved,Shawn,Reardon,shawn.reardon@umb.edu,617-921-7342,ON,PSU/CSU,2022-04-25 00:00:00,NaN,...,PSU/CSU,Tier II Monthly (ON) - PSU/CSU ($122.00),1CDC 13,NaN,NaN,302533,http://forms.umb.edu/upd/transportation/202204...,Y,Y,NaT
1,2022-04-26 00:00:00,Approved,Keri,O‚ÄôRiordan,Keri.ORiordan@umb.edu,781-635-4240,ON,PSU/CSU,2022-04-19 00:00:00,NaN,...,PSU/CSU,Tier II Monthly (ON) - PSU/CSU ($122.00),2HEY17,NaN,NaN,302449,http://forms.umb.edu/upd/transportation/7f7490...,Y,Y,NaT
2,2022-04-04 00:00:00,Approved,Mark,Cikacz,mark.cikacz@umb.edu,617-287-5000,ON,PSU/CSU,2022-04-04 00:00:00,NaN,...,PSU/CSU,Tier II Monthly (ON) - PSU/CSU ($122.00),423MV1,NaN,NaN,301599,http://forms.umb.edu/upd/transportation/mark_p...,Y,Y,Found
3,2022-03-24 00:00:00,Approved,Tim,Ford,timothy.ford@umb.edu,603-521-2921,ON,PSU/CSU,2022-04-01 00:00:00,NaN,...,PSU/CSU,Tier II Monthly (ON) - PSU/CSU ($122.00),4435530,4998609,NaN,282153,http://forms.umb.edu/upd/transportation/img_75...,Y,Y,Found
4,2022-03-22 00:00:00,Approved,Tyrone,Kindell,tyrone.kindell@umb.edu,617-287-5000,ON,PSU/CSU,2022-03-22 00:00:00,NaN,...,PSU/CSU,Tier II Monthly (ON) - PSU/CSU ($122.00),36VB75,NaN,NaN,290100,http://forms.umb.edu/upd/transportation/kailyn...,Y,Y,Found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798,2021-07-25 00:00:00,Approved,Rebecca,Romanow,Rebecca.Romanow@umb.edu,617-966-2677,ON,FSU,NaN,NaN,...,FSU,Semester (ON) - FSU ($550.00),P2222,NaN,NaN,279070,http://forms.umb.edu/upd/transportation/romano...,Y,Y,NaT
799,2021-07-22 00:00:00,Approved,Linda,Huang,Linda.Huang@umb.edu,617-287-6674,ON,FSU,NaN,NaN,...,FSU,Semester (ON) - FSU ($550.00),3ZP829,6032GY,NaN,246954,http://forms.umb.edu/upd/transportation/huang_...,Y,Y,Found
800,2021-07-21 00:00:00,Approved,Chi-kan,Hung,Richard.Hung@umb.edu,781-492-0596,ON,FSU,NaN,NaN,...,FSU,Semester (ON) - FSU ($550.00),6264NJ,7EXV10,NaN,265267,http://forms.umb.edu/upd/transportation/b5977e...,Y,Y,Found
801,2021-07-21 00:00:00,Approved,Michelle,Foster,Michelle.Foster@umb.edu,617-653-0096,ON,FSU,NaN,NaN,...,FSU,Semester (ON) - FSU ($550.00),4188 TN,NaN,NaN,246829,http://forms.umb.edu/upd/transportation/beacon...,Y,Y,Found


In [ ]:
dataframe_papers

,ID,Name
0,162750,"N Kamborian, Kristopher"
1,003194,I Schneider Graham
2,014864,"¥ Wade, Kahrim"
3,227636,"N Wise,Adam K."
4,Deduction,Total 222 -50
...,...,...
64,te,- an *
65,038548,"Ybarra,Raul E."
66,256364,"Zhu, Youxiang"
67,Deduction,"Total 9, 30375"
